In [1]:
import torch 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print('CUDA version: ', torch.version.cuda)
print('Default current GPU used: ', torch.cuda.current_device())
print('Device count: ', torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print('Device name:', torch.cuda.get_device_name(i))


if device.type == 'cuda':
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
CUDA version:  10.2
Default current GPU used:  0
Device count:  1
Device name: Tesla V100-PCIE-16GB
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
import logging

In [8]:
logging.basicConfig(filename='output.log',level = logging.INFO)

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from itertools import islice

ImportError: cannot import name '_imaging' from 'PIL' (/ccs/home/hstellar/.local/lib/python3.7/site-packages/PIL/__init__.py)

In [1]:
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from estimator import FEDformerEstimator

/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "
/ccs/proj/csc499/hstellar/rapids/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = get_dataset("electricity")

In [17]:
dataset

TrainDatasets(metadata=MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat_0', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24), train=DatasetCollection(datasets=[Map(data=JsonLinesFile(path=PosixPath('/ccs/home/hstellar/.mxnet/gluon-ts/datasets/electricity/train/data.json.gz')))], interleave=False), test=DatasetCollection(datasets=[Map(data=JsonLinesFile(path=PosixPath('/ccs/home/hstellar/.mxnet/gluon-ts/datasets/electricity/test/data.json.gz')))], interleave=False))

In [5]:
estimator = FEDformerEstimator(
    freq='h',
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length*7,
    dim_feedforward=16,
    num_feat_static_cat=1,
    cardinality=[321],
    embedding_dimension=[3],
    # attention hyper-params
    num_encoder_layers=2,
    num_decoder_layers=1,
    nhead=2,
    activation="relu",
    moving_avg=[24],
    # training params
    batch_size=128,
    num_batches_per_epoch=50,
    trainer_kwargs=dict(max_epochs=1, accelerator='gpu', gpus=1),
)

In [6]:
predictor = estimator.train(
    training_data=dataset.train,
    num_workers=8,
    # shuffle_buffer_length=1024
)


fourier enhanced block used!
modes=64, index=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 36, 37, 40, 41, 44, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 58, 59, 62, 63, 64, 65, 67, 68, 69, 70, 71, 73, 74, 76, 79, 80, 83]
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]
 fourier enhanced cross attention used!
dim_feedforward 16
enc_modes: 64, dec_modes: 54
encoder_self_att FourierBlock()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params
-----------------------------------------
0 | model | FEDformerModel | 263 K 
-----------------------------------------
263 K     Trainable params
0         Non-trainable params
263 K     Total params
1.053     Total estimated model params size (MB)


Epoch 0: : 0it [00:00, ?it/s]transformer_inputs torch.Size([128, 192, 50])
enc_input torch.Size([128, 168, 50])
torch.Size([128, 168, 50])
query proj torch.Size([128, 168, 50])
x_ft size torch.Size([128, 2, 25, 85])
weight size torch.Size([2, 25, 25, 64])
index [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 36, 37, 40, 41, 44, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 58, 59, 62, 63, 64, 65, 67, 68, 69, 70, 71, 73, 74, 76, 79, 80, 83]
torch.Size([128, 168, 50])
query proj torch.Size([128, 168, 50])
x_ft size torch.Size([128, 2, 25, 85])
weight size torch.Size([2, 25, 25, 64])
index [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 34, 36, 37, 40, 41, 44, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 58, 59, 62, 63, 64, 65, 67, 68, 69, 70, 71, 73, 74, 76, 79, 80, 83]
torch.Size([128, 24, 50])
query proj torch.Size([128, 24, 50])
x_ft size torch.Size([128, 2,

IndexError: index 13 is out of bounds for dimension 3 with size 13

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test, 
    predictor=predictor
)

In [ ]:
forecasts = list(forecast_it)

In [ ]:
tss = list(ts_it)

In [ ]:
evaluator = Evaluator()

In [ ]:
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))

In [ ]:
agg_metrics

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * dataset.metadata.prediction_length:], label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    plt.title(forecast.item_id)
    ax.xaxis.set_major_formatter(date_formater)

plt.gcf().tight_layout()
plt.legend()
plt.show()